In [1]:
#!touch 0-sequential.py
!chmod +x *.py

In [17]:
#!/usr/bin/env python3

#build_model = __import__('0-sequential').build_model
import tensorflow.keras as K

def build_model(nx, layers, activations, lambtha, keep_prob):
    """ doc """
    model = K.Sequential()
    
    dropped = K.layers.Dropout(rate=keep_prob)
    freg = K.regularizers.l2(l=lambtha)
    for i in range(len(layers)):
        model.add(K.layers.Dense(layers[i], input_dim=nx,
                                 activation=activations[i],
                                 kernel_regularizer=freg,
                                 activity_regularizer=dropped))
    return (model)

if __name__ == '__main__':
    network = build_model(784, [256, 256, 10], ['tanh', 'tanh', 'softmax'], 0.001, 0.95)
    network.summary()
    print(network.losses)

TypeError: __init__() got an unexpected keyword argument 'input_dim'